<a href="https://colab.research.google.com/github/yumin8136/Code1/blob/main/%E6%96%87%E6%9C%AC%E7%BF%BB%E8%AF%91%2Bstreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install streamlit -q

In [ ]:
!pip install --upgrade googletrans==4.0.0-rc1

In [ ]:
!pip install python-docx

In [101]:
%%writefile app.py
import streamlit as st
from googletrans import Translator
from io import BytesIO
from docx import Document
from docx.shared import Inches
import re

translator = Translator(service_urls=['translate.google.com'])

# 判断文本是否超过1000个字符，并找到第一个完整句子的位置
def split_text(text, max_length=1000):
    if len(text) <= max_length:
        return [text]

    sentence_end = r"[.!?]+"
    sentences = re.split(sentence_end, text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk + sentence) <= max_length:
            current_chunk += sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence

    # 如果最后一个chunk不为空，则添加到chunks中
    if current_chunk != "":
        chunks.append(current_chunk)

    return chunks


# 将Word文档保存为字节流
def save_docx_to_bytes(doc):
    doc_bytes = BytesIO()
    doc.save(doc_bytes)
    doc_bytes.seek(0)
    return doc_bytes.getvalue()


st.title("翻译程序")

# 上传文本文件
uploaded_file = st.file_uploader("上传文本文件", type=["txt"])

# 如果有上传的文件
if uploaded_file is not None:
    # 读取上传文件的内容
    content = uploaded_file.read().decode('utf-8')

    # 分割文本为适当大小的块
    chunks = split_text(content)

    # 创建左右两个列
    col1, col2 = st.columns(2)

    # 创建Word文档
    doc = Document()

    # 创建表格用于显示原文和翻译结果，并添加到Word文档
    for i, chunk in enumerate(chunks):
        translation = translator.translate(chunk, src='auto', dest='zh-CN')

        with col1:
            st.subheader("")
            st.text_area(" ", value=chunk, height=700)

        with col2:
            st.subheader("")
            st.text_area(" ", value=translation.text, height=700)

        # 将原文和翻译结果添加到Word文档中的表格
        table = doc.add_table(rows=1, cols=2)
        table.autofit = False
        table.columns[0].width = Inches(3)
        table.columns[1].width = Inches(3)
        table.style = 'Table Grid'
        table.allow_autofit = False

        first_cell = table.cell(0, 0)
        first_cell.text = chunk

        second_cell = table.cell(0, 1)
        second_cell.text = translation.text

        # 添加分页符
        if i != len(chunks) - 1:
            doc.add_page_break()

    # 添加"下载成Word"按钮
    st.download_button(
        label="下载成Word",
        data=save_docx_to_bytes(doc),
        file_name="translation.docx",
        mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
    )





Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.170.4:8501

npx: installed 22 in 2.61s
your url is: https://three-carpets-kiss.loca.lt
